In [10]:
import numpy as np
import pandas as pd
import optuna
import seaborn as sns
import lightgbm as lgb
import joblib

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

from sklearn.model_selection import cross_val_score, KFold, TimeSeriesSplit

In [18]:
train_df = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')

cv_indicies = list(KFold().split(train_df))

In [19]:
#check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(train_df.shape))
print("The test data size before dropping Id feature is : {} ".format(test_df.shape))

#Save the 'Id' column
train_ID = train_df['Id']
test_ID = test_df['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train_df.drop("Id", axis = 1, inplace = True)
test_df.drop("Id", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train_df.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test_df.shape))

#all_data = pd.concat([train_df,test_df])

The train data size before dropping Id feature is : (1460, 81) 
The test data size before dropping Id feature is : (1459, 80) 

The train data size after dropping Id feature is : (1460, 80) 
The test data size after dropping Id feature is : (1459, 79) 


In [20]:
train_df = train_df.fillna('None')
test_df = test_df.fillna('None')

In [21]:
X_train = train_df.drop(['SalePrice'],axis=1, inplace = False)
y_train = train_df['SalePrice']

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSSubClass     1460 non-null   int64 
 1   MSZoning       1460 non-null   object
 2   LotFrontage    1460 non-null   object
 3   LotArea        1460 non-null   int64 
 4   Street         1460 non-null   object
 5   Alley          1460 non-null   object
 6   LotShape       1460 non-null   object
 7   LandContour    1460 non-null   object
 8   Utilities      1460 non-null   object
 9   LotConfig      1460 non-null   object
 10  LandSlope      1460 non-null   object
 11  Neighborhood   1460 non-null   object
 12  Condition1     1460 non-null   object
 13  Condition2     1460 non-null   object
 14  BldgType       1460 non-null   object
 15  HouseStyle     1460 non-null   object
 16  OverallQual    1460 non-null   int64 
 17  OverallCond    1460 non-null   int64 
 18  YearBuilt      1460 non-null

In [22]:
display(train_df)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,20,RL,80,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,60,RL,68,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,70,RL,60,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,60,RL,84,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,None,None,None,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62,7917,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,8,2007,WD,Normal,175000
1456,20,RL,85,13175,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,2,2010,WD,Normal,210000
1457,70,RL,66,9042,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68,9717,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,4,2010,WD,Normal,142125


In [23]:
display(test_df)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80,11622,Pave,None,Reg,Lvl,AllPub,Inside,...,120,0,None,MnPrv,None,0,6,2010,WD,Normal
1,20,RL,81,14267,Pave,None,IR1,Lvl,AllPub,Corner,...,0,0,None,None,Gar2,12500,6,2010,WD,Normal
2,60,RL,74,13830,Pave,None,IR1,Lvl,AllPub,Inside,...,0,0,None,MnPrv,None,0,3,2010,WD,Normal
3,60,RL,78,9978,Pave,None,IR1,Lvl,AllPub,Inside,...,0,0,None,None,None,0,6,2010,WD,Normal
4,120,RL,43,5005,Pave,None,IR1,HLS,AllPub,Inside,...,144,0,None,None,None,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21,1936,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,6,2006,WD,Normal
1455,160,RM,21,1894,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,4,2006,WD,Abnorml
1456,20,RL,160,20000,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,9,2006,WD,Abnorml
1457,85,RL,62,10441,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,MnPrv,Shed,700,7,2006,WD,Normal


In [15]:
print(train_df.shape)
print(test_df.shape)

(1460, 81)
(1459, 80)


In [11]:
#modelセット
model = lgb.LGBMRegressor(colsample_bytree=0.68, 
                              learning_rate=0.1, 
                              min_child_samples=40, 
                              n_estimators=183, 
                              num_leaves=9064, 
                              reg_alpha=0.13, 
                              reg_lambda=0.0, 
                              subsample=0.83, 
                              subsample_for_bin=155127, 
                              subsample_freq=214953,
                              n_jobs=4, random_state=1)

#cross_val_score(estimator=model,X=df,y=df_target,cv=cv_indicies)

In [20]:
def objective(trial):
    
    t_num_leaves = trial.suggest_int("num_leaves", 20, 10000)
    t_learning_rate = trial.suggest_float("learning_rate", 0.1, 0.7,step=0.1)
#    t_learning_rate = round(t_learning_rate,1)
    t_n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    t_subsample_for_bin = trial.suggest_int("subsample_for_bin", 0, 300000)
    t_subsample_freq = trial.suggest_int("subsample_freq", 0, 300000)
    t_reg_alpha = trial.suggest_float("reg_alpha", 0.1, 0.2,step = 0.01)
#    t_reg_alpha = round(t_reg_alpha,2)
    t_reg_lambda = trial.suggest_float("reg_lambda", 0, 0.2,step = 0.01)
#    t_reg_lambda = round(t_reg_lambda,2)
    t_colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 1,step = 0.01)
#    t_colsample_bytree = round(t_colsample_bytree,2)
    t_subsample = trial.suggest_float("subsample", 0.01, 1,step = 0.01)
#    t_subsample = round(t_subsample,2)
    t_min_child_samples = trial.suggest_int("min_child_samples", 0, 100)
    
    
    lgb_param = {"boosting_type":"gbdt",
                 "num_leaves":t_num_leaves,
                 "learning_rate":t_learning_rate,
                 "subsample_for_bin":t_subsample_for_bin,
                 "subsample_freq":t_subsample_freq,
                 "reg_alpha":t_reg_alpha,
                 "reg_lambda":t_reg_lambda,
                 "colsample_bytree":t_colsample_bytree,
                 "subsample":t_subsample,
                 "min_child_samples":t_min_child_samples,
                 "n_estimators":t_n_estimators
                }
    
    model = lgb.LGBMRegressor(
                **lgb_param,
                n_jobs=4, random_state=1)
    
    score = cross_val_score(estimator=model,X=X_train,y=y_train,cv=cv_indicies)[0]
    return score

study = optuna.create_study(direction='maximize',
                            study_name='Hyperparameters',
                            storage='sqlite:///optuna_studies.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=100)

[I 2022-03-07 19:18:43,693] Using an existing study with name 'Hyperparameters' instead of creating a new one.
c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\p

c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=

c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=

c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=

c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=

...

c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=

c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=

c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_model)
  File "c:\users\used\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py", line 228, in train
    booster = Booster(params=

[W 2022-03-07 19:18:46,944] Trial 70 failed, because the objective function returned nan.


KeyboardInterrupt: 

In [ ]:
param_b = study.best_params
param = {key: value for key, value in param_b.items()
            if key in {'colsample_bytree',
                       'learning_rate',
                       'min_child_samples',
                       'n_estimators',
                       'num_leaves',
                       'reg_alpha',
                       'reg_lambda',
                       'subsample',
                       'subsample_for_bin',
                       'subsample_freq'}
      }

In [ ]:
model = lgb.LGBMRegressor(
            **param,
            n_jobs=4, random_state=1)

model.fit(train_df, test_df)
joblib.dump(model, 'model.xz', compress=True)

importance = pd.DataFrame(sorted(zip(model.feature_importances_,df_head)), columns=['Value','Feature'])
display(importance)